In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import find_peaks
from scipy.optimize import curve_fit

import datetime
import os
import shutil

from glob import glob
from copy import deepcopy

In [4]:

res = np.arange(100,320,20)  # Re 100-300 in steps of 20
bcs = ["hwbbc2", "ibb1c2"]

base_path = "/home/mbille/Documents/cluster_hbrs_home/02_lbm_simulations/data_231031_c2d_compare3D_fineRe100-300_GPD30_DpY10_BGK_T300"

paths_dict = dict() # path matches
res_dict = dict() # lists of available gpds

# create path_dict: keys= bc_col Combination, values: lists of paths to gpd-variants
for bc in bcs:
    paths_dict[bc] = []
    res_dict[bc] = []
    for re in res:
        dir_names = glob(base_path+"/*c2d_compare3D_fineRe100-300_GPD30_DpY10_BGK_T300_"+bc+"_Re"+str(re)+"_*")
        if len(dir_names)>1:
            print("(!) Warning: more than 1 simulation for:", bc+"_Re"+str(re))
            print(dir_names)
        elif len(dir_names) == 1:
            # this is what we want: one unique simulation found
            paths_dict[bc].append(*dir_names)
            res_dict[bc].append(re)

#        print(base_path+"/*c2d_compare3D_fineRe100-300_GPD30_DpY10_BGK_T300_"+bc+"_Re"+str(re)+"_*")
#        print(bc+"_Re"+str(re)+"_*:",dir_names)
#        print(paths_dict[bc])

#print(gpds)

In [7]:
# FIT SINE WAVE -> STROUHAL NUMBER
#res = np.arange(100,320,20)  # Re 100-300 in steps of 20
bcs = ["hwbbc2", "ibb1c2"]

filename = "/lift_coefficient.txt"
periodic_start=0.9

# function to fit
def sine_func(xx,a,b,c,d):
    return a*np.sin(2*np.pi*b*xx+c)+d

for bc in bcs:
    lift_St_fit = np.zeros((2,len(res_dict[bc])))
    lift_St_fit[0] = res_dict[bc]
    for re_i in range(len(lift_St_fit[0])):
        # load timeseries
        lift_timeseries = np.loadtxt(paths_dict[bc][re_i]+filename)
        ph=periodic_start
        if res_dict[bc][re_i] == 300:
            periodic_start=0.95
        lift_converged = lift_timeseries[int(lift_timeseries.shape[0]*periodic_start-1):]
        periodic_start=ph

        # fit sine_wave
        coefficients, sheeesh = curve_fit(sine_func, lift_converged[:,1], lift_converged[:,2], p0=(0.7,0.2,0.5,0))

        # plot fit and save to Desktop/St_Fits/...
        fig, ax = plt.subplots(constrained_layout=True)
        plt.plot(lift_converged[:,1], lift_converged[:,2], lift_converged[:,1], sine_func(lift_converged[:,1], *coefficients))
        plt.legend(["lift", "fit"])
        ax.set_xlabel("physical time / s")
        ax.set_ylabel("Coefficient of Lift")
        ax.set_ylim([-1,1])
        plt.savefig("/home/mbille/Desktop/St_Fits/St_LiftFit_2D_"+bc+"_Re"+str(res_dict[bc][re_i])+".png")
        plt.close(fig)

        # store St
        lift_St_fit[1,re_i]=coefficients[1]
    np.savetxt("/home/mbille/Desktop/St_Fits/St_LiftFit_2D_"+bc+"-St(Re).txt", lift_St_fit, header="2D, DpY19, D2Q9, "+bc)

In [8]:
# AVERAGE PEAKS OF LIFT -> Cl_max(mean)

#bcs= ["fwbb"]
#cols=["kbc"]

filename = "/lift_coefficient.txt"
periodic_start=0.75

for bc in bcs:
    lift_Cl_maxmean = np.zeros((2,len(res_dict[bc])))
    lift_Cl_maxmean[0] = res_dict[bc]
    for re_i in range(len(lift_Cl_maxmean[0])):
        # load timeseries
        lift_timeseries = np.loadtxt(paths_dict[bc][re_i]+filename)
        lift_converged = lift_timeseries[int(lift_timeseries.shape[0]*periodic_start-1):]

        # find peaks
        #peaks_max = find_peaks(lift_converged[:,2], prominence=lift_converged[:,2].max())  # prominence: "the minimum height necessary to descend to get from the summit to any higher terrain"
        peaks_max = find_peaks(lift_converged[:,2], prominence=lift_converged[:,2].max()-(lift_converged[:,2].max()+lift_converged[:,2].min())*0.5)
        lift_Cl_maxmean[1,re_i] = lift_converged[peaks_max[0],2].mean()

        # plot fit and save to Desktop/St_Fits/...
        fig, ax = plt.subplots(constrained_layout=True)
        ax.plot(lift_timeseries[:,1], lift_timeseries[:,2])
        plt.scatter(lift_converged[peaks_max[0],1],lift_converged[peaks_max[0],2],marker="+", facecolors="r")
        plt.axhline(y=lift_Cl_maxmean[1,re_i], color="r", ls="--", lw=0.5)
        plt.legend(["lift", "Cl_maxmean"])
        ax.set_xlabel("physical time / s")
        ax.set_ylabel("Coefficient of Lift")
        ax.set_ylim([-1,1])
        plt.savefig("/home/mbille/Desktop/Cl_Peaks/Cl_Peaks_2D_"+bc+"_Re"+str(res_dict[bc][re_i])+".png")
        plt.close(fig)
    np.savetxt("/home/mbille/Desktop/Cl_Peaks/Cl_Peaks_2D_"+bc+"-Cl(Re).txt", lift_Cl_maxmean, header="2D, DpY19, D2Q9, "+bc)

In [35]:
lift_converged[:,2].min()

-0.7215750477431399

In [34]:
lift_converged[:,2].max()

0.7213720431780384